# Notes

**See Y's notebook at `/n/data1/hms/dbmi/zitnik/lab/users/yeh803/PLM/raw_data//n/data1/hms/dbmi/zitnik/lab/users/yeh803/PLM/raw_data/test.ipynb` with complete 

Owen has processed string - results available here:
Code: [https://github.com/mims-harvard/ProteinLanguageModel/commit/686ce1045551d37291a618df69e5a5d8e5faad5f]
Notes: 
- Evidence scores are multiplied by 1000 in STRING 
  - [Source](https://string-db.org/cgi/help?sessionId=bqbnNFiL0skB) (search 1000), cutoff of 0.7 on total score also mentioned here
  - Better future approach, take different disjunctive thresholds (for threshold values - search for 0.4 [here](https://string-db.org/help//getting_started/#starting-point))
  - [How are scores computed?](https://string-db.org/help//faq/#how-are-the-scores-computed)
- Currently uses all evidence cols, we need to decide which ones to use (explanations [here](https://string-db.org/help//getting_started/#evidence))
  - We probably only functional ones, Yepeng thinks:
    - Coexpression
    - Datasets
    - Experiments
- Uses the DSs in `torchprotein`
- `Seq` is a wrapper for string, defined in Bio libary

In [1]:
import os
import torch
import pandas as pd
import numpy as np
from tqdm import trange

from Bio import SeqIO

In [4]:
# Load owen's work on data
data = torch.load('/n/data1/hms/dbmi/zitnik/lab/users/yeh803/PLM/raw_data/string/processed_string/processed_seqs_small.pt')

# Analysing STRING data

#### Helpers

In [37]:
def count_nodes(df):
    return pd.concat([df['protein1'], df['protein2']]).nunique()

def filter_by_special_average(association_table, threshold):
    prior = 0.041
    relation_scores_no_prior = []
    for relation in ['coexpression', 'experimental', 'database']:
        relation_scores_no_prior.append((association_table[relation] - prior) / (1 - prior))
    combined_score_no_prior = 1 - (1 - pd.concat(relation_scores_no_prior, axis=1)).prod(axis=1)
    combined_score = combined_score_no_prior + prior * (1 - combined_score_no_prior)
    return(association_table[combined_score > threshold])

#### Load and Preprocess Dataframe

In [77]:
data_path = '/n/data1/hms/dbmi/zitnik/lab/users/yeh803/PLM/raw_data/'
detailed_data_filename = 'string/9606.protein.links.detailed.v11.5.txt'

detailed_associations = pd.read_table(os.path.join(data_path, detailed_data_filename), sep=' ')

# Divide scores by 1000
detailed_associations.iloc[:, 2:] = detailed_associations.iloc[:, 2:] / 1000.0

# Remove "9606." (human) prefixes
detailed_associations['protein1'] = detailed_associations['protein1'].str.replace('9606.', '')
detailed_associations['protein2'] = detailed_associations['protein2'].str.replace('9606.', '')

# Drop unwanted columns
detailed_associations = detailed_associations.drop(columns=['neighborhood', 'fusion', 'cooccurence', 'textmining', 'combined_score'])

detailed_associations

/tmp/ipykernel_28424/2650170938.py:10: FutureWarning: The default value of regex will change from True to False in a future version.
  detailed_associations['protein1'] = detailed_associations['protein1'].str.replace('9606.', '')
/tmp/ipykernel_28424/2650170938.py:11: FutureWarning: The default value of regex will change from True to False in a future version.
  detailed_associations['protein2'] = detailed_associations['protein2'].str.replace('9606.', '')


,protein1,protein2,coexpression,experimental,database
0,ENSP00000000233,ENSP00000379496,0.054,0.000,0.0
1,ENSP00000000233,ENSP00000314067,0.000,0.180,0.0
2,ENSP00000000233,ENSP00000263116,0.062,0.152,0.0
3,ENSP00000000233,ENSP00000361263,0.000,0.161,0.0
4,ENSP00000000233,ENSP00000409666,0.082,0.213,0.0
...,...,...,...,...,...
11938493,ENSP00000485678,ENSP00000354800,0.213,0.000,0.0
11938494,ENSP00000485678,ENSP00000308270,0.152,0.000,0.0
11938495,ENSP00000485678,ENSP00000335660,0.182,0.000,0.0
11938496,ENSP00000485678,ENSP00000300127,0.155,0.000,0.0


In [ ]:
print(f'Total associations: {len(detailed_associations):,}')
print(f'    Nodes involved: {count_nodes(detailed_associations):,}')

In [42]:
# detailed_associations_filter_by_combined_score = detailed_associations[detailed_associations['combined_score'] > 0.4]
# print(f'Associations with combined_score > 0.4: {len(detailed_associations_filter_by_combined_score):,}')
# print(f'    Nodes remaining: {count_nodes(detailed_associations_filter_by_combined_score):,}')

Associations with combined_score > 0.4: 1,784,580
    Nodes remaining: 19,302


In [41]:
# detailed_associations_filter_by_combined_score = detailed_associations[detailed_associations['combined_score'] > 0.7]
# print(f'Associations with combined_score > 0.7: {len(detailed_associations_filter_by_combined_score):,}')
# print(f'    Nodes remaining: {count_nodes(detailed_associations_filter_by_combined_score):,}')

Associations with combined_score > 0.7: 504,026
    Nodes remaining: 16,795


In [34]:
detailed_associations_filter_by_disjunction_04 = detailed_associations[(detailed_associations['coexpression'] > 0.4) | (detailed_associations['experimental'] > 0.4) | (detailed_associations['database'] > 0.4)]
print(f'Associations with disjunctive score > 0.4: {len(detailed_associations_filter_by_disjunction_04):,}')
print(f'    Nodes remaining: {count_nodes(detailed_associations_filter_by_disjunction_04):,}')

Associations with disjunctive score > 0.4: 651,506
    Nodes remaining: 15,542


In [35]:
detailed_associations_filter_by_disjunction_07 = detailed_associations[(detailed_associations['coexpression'] > 0.7) | (detailed_associations['experimental'] > 0.7) | (detailed_associations['database'] > 0.7)]
print(f'Associations with disjunctive score > 0.7: {len(detailed_associations_filter_by_disjunction_07):,}')
print(f'    Nodes remaining: {count_nodes(detailed_associations_filter_by_disjunction_07):,}')

Associations with disjunctive score > 0.7: 293,224
    Nodes remaining: 11,623



**Yepeng also has implemented the following analysis using STRING's [special average](https://string-db.org/help//faq/#how-are-the-scores-computed)** (not sure what the proper name for it is):

In [40]:
detailed_associations_filter_by_special_average = filter_by_special_average(detailed_associations, 0.4)
print(f'Associations with special average > 0.4: {len(detailed_associations_filter_by_special_average):,}')
print(f'    Nodes remaining: {count_nodes(detailed_associations_filter_by_special_average):,}')

Associations with special average > 0.4: 656,540
    Nodes remaining: 15,234


In [39]:
detailed_associations_filter_by_special_average = filter_by_special_average(detailed_associations, 0.7)
print(f'Associations with special average > 0.7: {len(detailed_associations_filter_by_special_average):,}')
print(f'    Nodes remaining: {count_nodes(detailed_associations_filter_by_special_average):,}')

Associations with special average > 0.7: 313,844
    Nodes remaining: 11,790


## Filter STRING data

In [73]:
# Filter rows with evidence score below threshold (using special average)
THRESHOLD = 0.4
detailed_associations_filtered = filter_by_special_average(detailed_associations, THRESHOLD)
display(detailed_associations)
print(f'Associations with special average > 0.7: {len(detailed_associations_filtered):,}')
print(f'    Nodes remaining: {count_nodes(detailed_associations_filtered):,}')

,protein1,protein2,neighborhood,fusion,cooccurence,coexpression,experimental,database,textmining,combined_score
0,ENSP00000000233,ENSP00000379496,0.0,0.0,0.0,0.054,0.000,0.0,0.144,0.155
1,ENSP00000000233,ENSP00000314067,0.0,0.0,0.0,0.000,0.180,0.0,0.061,0.197
2,ENSP00000000233,ENSP00000263116,0.0,0.0,0.0,0.062,0.152,0.0,0.101,0.222
3,ENSP00000000233,ENSP00000361263,0.0,0.0,0.0,0.000,0.161,0.0,0.064,0.181
4,ENSP00000000233,ENSP00000409666,0.0,0.0,0.0,0.082,0.213,0.0,0.072,0.270
...,...,...,...,...,...,...,...,...,...,...
11938493,ENSP00000485678,ENSP00000354800,0.0,0.0,0.0,0.213,0.000,0.0,0.000,0.213
11938494,ENSP00000485678,ENSP00000308270,0.0,0.0,0.0,0.152,0.000,0.0,0.000,0.151
11938495,ENSP00000485678,ENSP00000335660,0.0,0.0,0.0,0.182,0.000,0.0,0.000,0.181
11938496,ENSP00000485678,ENSP00000300127,0.0,0.0,0.0,0.155,0.000,0.0,0.000,0.154


Associations with special average > 0.7: 656,540
    Nodes remaining: 15,234


## Map ENSP IDs to UniProt IDs

In [85]:
# Find all remaining proteins (proteins involved in a relation), using ENSP ids
remaining_ensps = set(pd.concat([detailed_associations_filtered.protein1, detailed_associations_filtered.protein2]).unique())

# Generate mapping of ENSP id to sequence
parser = SeqIO.parse(data_path + "/string/9606.protein.sequences.v11.5.fa", 'fasta')
ENSP_to_sequence_dict = {record.id.replace("9606.", "") : str(record.seq) for record in parser if record.id.replace("9606.", "") in remaining_ensps}  # NOTE: if dealing with full STRING, load all sequences and use pandas to filter instead

Mapping file below generated using BioMart - [link to download mapping file](https://www.ensembl.org/biomart/martview/3912bde0f7f4e26e93af76b712b4b85f).

**TODO: Some IDs are deprecated / no longer used in database, these come up as `unmapped`, since the mapping file does not include them.**

In [117]:
string_mapping_table = pd.read_table(os.path.join(data_path, 'string/9606.protein.aliases.v11.5.txt'))
string_mapping_table['#string_protein_id'] = string_mapping_table['#string_protein_id'].str.replace('9606.', '', regex=False)
string_mapping_table

unmapped = set(remaining_ensps) - set(string_mapping_table['#string_protein_id'])
print(f'Number of rows in mapping: {len(string_mapping_table):,}')
print(f'Number of unmapped proteins in detailed_associations_filtered: {len(unmapped):,}')

string_mapping_table

Number of rows in mapping: 4,213,391
Number of unmapped proteins in detailed_associations_filtered: 0


,#string_protein_id,alias,source
0,ENSP00000000233,2B6H,BLAST_UniProt_DR_PDB
1,ENSP00000000233,2B6H,Ensembl_HGNC_UniProt_ID(supplied_by_UniProt)_D...
2,ENSP00000000233,2B6H,Ensembl_PDB
3,ENSP00000000233,2B6H,Ensembl_UniProt_DR_PDB
4,ENSP00000000233,381,BLAST_KEGG_GENEID
...,...,...,...
4213386,ENSP00000485678,hsa:219952,BLAST_KEGG_KEGGID
4213387,ENSP00000485678,olfactory receptor family 6 subfamily Q member 1,Ensembl_HGNC_Approved_Name
4213388,ENSP00000485678,uc010rjz.2,BLAST_UniProt_DR_UCSC
4213389,ENSP00000485678,uc010rjz.2,Ensembl_HGNC_UCSC_ID(supplied_by_UCSC)


In [123]:
string_mapping_table[string_mapping_table['source'].str.contains('Ensembl_HGNC_UniProt_ID')]

,#string_protein_id,alias,source
1,ENSP00000000233,2B6H,Ensembl_HGNC_UniProt_ID(supplied_by_UniProt)_D...
8,ENSP00000000233,381,Ensembl_HGNC_UniProt_ID(supplied_by_UniProt)_D...
21,ENSP00000000233,ADP-ribosylation factor 5,Ensembl_HGNC_UniProt_ID(supplied_by_UniProt)_D...
33,ENSP00000000233,ARF5,Ensembl_HGNC_UniProt_ID(supplied_by_UniProt)_GN
57,ENSP00000000233,ARF5_HUMAN,Ensembl_HGNC_UniProt_ID(supplied_by_UniProt)_ID
...,...,...,...
4213380,ENSP00000485678,Q6IFH1,Ensembl_HGNC_UniProt_ID(supplied_by_UniProt)_AC
4213382,ENSP00000485678,Q8NGQ2,Ensembl_HGNC_UniProt_ID(supplied_by_UniProt)
4213383,ENSP00000485678,Q8NGQ2,Ensembl_HGNC_UniProt_ID(supplied_by_UniProt)_AC
4213385,ENSP00000485678,Q96R34,Ensembl_HGNC_UniProt_ID(supplied_by_UniProt)_AC


In [129]:
string_mapping_table[string_mapping_table['source'].str.contains('UniProt')].source.unique()

array(['BLAST_UniProt_DR_PDB',
       'Ensembl_HGNC_UniProt_ID(supplied_by_UniProt)_DR_PDB',
       'Ensembl_UniProt_DR_PDB', 'BLAST_UniProt_DR_GeneID',
       'Ensembl_HGNC_UniProt_ID(supplied_by_UniProt)_DR_GeneID',
       'Ensembl_UniProt_DR_GeneID', 'Ensembl_UniProt',
       'BLAST_UniProt_DE_RecName_Full',
       'Ensembl_HGNC_UniProt_ID(supplied_by_UniProt)_DE_RecName',
       'Ensembl_UniProt_DE_RecName', 'BLAST_UniProt_GN_Name',
       'Ensembl_HGNC_UniProt_ID(supplied_by_UniProt)_GN',
       'Ensembl_UniProt_GN', 'BLAST_UniProt_ID',
       'Ensembl_HGNC_UniProt_ID(supplied_by_UniProt)_ID',
       'Ensembl_UniProt_ID', 'BLAST_UniProt_DR_RefSeq',
       'Ensembl_HGNC_UniProt_ID(supplied_by_UniProt)_DR_RefSeq',
       'Ensembl_UniProt_DR_RefSeq', 'BLAST_UniProt_DR_neXtProt',
       'Ensembl_HGNC_UniProt_ID(supplied_by_UniProt)_DR_neXtProt',
       'Ensembl_UniProt_DR_neXtProt', 'BLAST_UniProt_AC',
       'Ensembl_HGNC_UniProt_ID(supplied_by_UniProt)_AC',
       'Ensembl_UniProt_A

In [157]:
# Path to mapping from ENSP ID to UniProt / other IDs
# TODO: update mapping
ensp_to_uniprot_path = '/n/data1/hms/dbmi/zitnik/lab/users/yeh803/PLM/Ensembl_ensp_uniprot_reactome.txt'
ensp_to_uniprot_table = pd.read_table(ensp_to_uniprot_path)

# Drop unnecessary cols
ensp_to_uniprot_table = ensp_to_uniprot_table[['Protein stable ID', 'UniProtKB/Swiss-Prot ID']]

# TODO: improve (no iteration)
ensp_to_uniprot_dict = {row['Protein stable ID'] : row['UniProtKB/Swiss-Prot ID'] for _, row in ensp_to_uniprot_table.iterrows()} 
print(len(ensp_to_uniprot_dict))

ensp_to_uniprot_unmapped = set(remaining_ensps) - set(ensp_to_uniprot_table['Protein stable ID'])
print(f'Number of rows in mapping: {len(ensp_to_uniprot_table):,}')
print(f'Number of unmapped proteins in detailed_associations_filtered: {len(ensp_to_uniprot_unmapped):,}')

121541
Number of rows in mapping: 429,218
Number of unmapped proteins in detailed_associations_filtered: 570


In [165]:
print(list(ensp_to_uniprot_dict.items())[:10])

[('ENSP00000354687', 'P03886'), ('ENSP00000355046', 'P03891'), ('ENSP00000354499', 'P00395'), ('ENSP00000354876', 'P00403'), ('ENSP00000355265', 'P03928'), ('ENSP00000354632', 'P00846'), ('ENSP00000354982', 'P00414'), ('ENSP00000355206', 'P03897'), ('ENSP00000354728', 'P03901'), ('ENSP00000354961', 'P03905')]


In [160]:
# TODO: remove - fix unmapped proteins rather than filtering out 
detailed_associations_filtered = detailed_associations_filtered[~detailed_associations_filtered['protein1'].isin(unmapped)]
detailed_associations_filtered = detailed_associations_filtered[~detailed_associations_filtered['protein2'].isin(unmapped)]

# Convert associations table to use uniprot IDs
detailed_associations_filtered['protein1_uniprot'] = detailed_associations_filtered['protein1'].apply(lambda ensp: ensp_to_uniprot_dict[ensp])
detailed_associations_filtered['protein2_uniprot'] = detailed_associations_filtered['protein2'].apply(lambda ensp: ensp_to_uniprot_dict[ensp])
detailed_associations_filtered

KeyError: 'ENSP00000035383'

In [149]:
# seqs = pd.read_table(os.path.join(data_path, '/string/9606.protein.sequences.v11.5.fa'), sep=' ')
# seqs

Attempted mapping in reverse direction: looks bad on first glance but remember that PKG25 includes a lot of non-human protiens

In [147]:
# Mapping in reverse direction: from proteins in PKG25 to aliases in STRING mapping file
pkg25_protein2id = pd.read_table(os.path.join(data_path, 'ProteinKG25/protein2id.txt'), header=None, sep=' ')
pkg25_protein2id = pkg25_protein2id.rename(columns={0: 'protein_uniprot_id', 1: 'protein_pkg25_id'})

unmapped_pkg25_string = set(pkg25_protein2id['protein_uniprot_id']) - set(string_mapping_table['alias'])
print(len(pkg25_protein2id))
print(len(unmapped_pkg25_string))
unmapped_pkg25_string

565254
546254


In [81]:
all_ensps_string

{'ENSP00000280772',
 'ENSP00000420321',
 'ENSP00000264318',
 'ENSP00000341662',
 'ENSP00000216144',
 'ENSP00000363654',
 'ENSP00000261866',
 'ENSP00000358867',
 'ENSP00000252136',
 'ENSP00000346694',
 'ENSP00000313885',
 'ENSP00000315775',
 'ENSP00000322594',
 'ENSP00000228506',
 'ENSP00000265421',
 'ENSP00000365233',
 'ENSP00000418770',
 'ENSP00000302150',
 'ENSP00000355593',
 'ENSP00000406598',
 'ENSP00000455785',
 'ENSP00000263088',
 'ENSP00000360538',
 'ENSP00000476312',
 'ENSP00000379256',
 'ENSP00000439189',
 'ENSP00000311280',
 'ENSP00000258243',
 'ENSP00000368767',
 'ENSP00000401018',
 'ENSP00000331019',
 'ENSP00000334115',
 'ENSP00000325917',
 'ENSP00000276914',
 'ENSP00000481721',
 'ENSP00000263239',
 'ENSP00000353549',
 'ENSP00000376708',
 'ENSP00000482957',
 'ENSP00000428220',
 'ENSP00000438492',
 'ENSP00000248594',
 'ENSP00000276052',
 'ENSP00000311997',
 'ENSP00000321927',
 'ENSP00000335025',
 'ENSP00000279242',
 'ENSP00000332473',
 'ENSP00000419101',
 'ENSP00000245908',


## From Owen's code - not in use

In [63]:
def get_string_data(
    df,
    confidence_thresh = None, # Threshold links at this threshold and above
    ):
        
    if confidence_thresh is not None:
        df = filter_by_special_average(df, confidence_thresh)
    
    display(df)
    return

    # Sequences extraction:
    seqfind = SeqIO.to_dict(SeqIO.parse(os.path.join(path, '9606.protein.sequences.v11.5.fa'), 'fasta'))

    # for i in range(1):
    #     print('Sample {}:'.format(i))
    #     s1 = df.iloc[i,0]
    #     s2 = df.iloc[i,1]
    #     print('ID 1: {} \t - Seq = {}'.format(s1, seqfind[s1].seq))
    #     print('ID 2: {} \t - Seq = {}'.format(s2, seqfind[s2].seq))
        # print('')
        # print(seqfind[s1].seq)

    all_seqs = []
    all_targets = []

    skipped_pairs = 0
    for i in trange(df.shape[0]):
        try:
            s1, s2 = df.iloc[i,0], df.iloc[i,1]
            all_seqs.append([str(seqfind[s1].seq), str(seqfind[s2].seq)])

            all_targets.append(df.iloc[i,2:].to_numpy())
        except KeyError:
            skipped_pairs += 1
            continue # Don't add pair if we can't find the sequence

    print('Done')
    print(f'Skipped {skipped_pairs} pairs.')
    return all_seqs, np.stack(all_targets)

In [64]:
# Confidence levels in links:
# low = >0.15
# medium = 0.4
# high = 0.7
# highest = 0.9

all_seq, all_targ = get_string_data(detailed_associations, 0.7)


,protein1,protein2,neighborhood,fusion,cooccurence,coexpression,experimental,database,textmining,combined_score
132,9606.ENSP00000000233,9606.ENSP00000262812,0.0,0.0,0.00,0.190,0.163,0.6,0.173,0.745
145,9606.ENSP00000000233,9606.ENSP00000449270,0.0,0.0,0.00,0.162,0.210,0.6,0.194,0.757
160,9606.ENSP00000000233,9606.ENSP00000263245,0.0,0.0,0.00,0.063,0.391,0.6,0.527,0.877
187,9606.ENSP00000000233,9606.ENSP00000440005,0.0,0.0,0.05,0.099,0.679,0.9,0.059,0.969
214,9606.ENSP00000000233,9606.ENSP00000325002,0.0,0.0,0.00,0.062,0.315,0.6,0.215,0.771
...,...,...,...,...,...,...,...,...,...,...
11938190,9606.ENSP00000485663,9606.ENSP00000346001,0.0,0.0,0.00,0.738,0.000,0.0,0.158,0.770
11938211,9606.ENSP00000485663,9606.ENSP00000248342,0.0,0.0,0.00,0.699,0.996,0.9,0.957,0.999
11938226,9606.ENSP00000485663,9606.ENSP00000416255,0.0,0.0,0.00,0.167,0.808,0.0,0.473,0.908
11938234,9606.ENSP00000485663,9606.ENSP00000220849,0.0,0.0,0.00,0.907,0.993,0.9,0.896,0.999


TypeError: cannot unpack non-iterable NoneType object

In [48]:

# print('AS', len(all_seq))
# print('AT', all_targ.shape)

# torch.save(all_seq, 'processed_seqs.pt')
# torch.save(all_targ, 'processed_targets.pt')

,protein1,protein2,neighborhood,fusion,cooccurence,coexpression,experimental,database,textmining,combined_score
0,9606.ENSP00000000233,9606.ENSP00000379496,0.0,0.0,0.0,0.054,0.000,0.0,0.144,0.155
1,9606.ENSP00000000233,9606.ENSP00000314067,0.0,0.0,0.0,0.000,0.180,0.0,0.061,0.197
2,9606.ENSP00000000233,9606.ENSP00000263116,0.0,0.0,0.0,0.062,0.152,0.0,0.101,0.222
3,9606.ENSP00000000233,9606.ENSP00000361263,0.0,0.0,0.0,0.000,0.161,0.0,0.064,0.181
4,9606.ENSP00000000233,9606.ENSP00000409666,0.0,0.0,0.0,0.082,0.213,0.0,0.072,0.270
...,...,...,...,...,...,...,...,...,...,...
11938493,9606.ENSP00000485678,9606.ENSP00000354800,0.0,0.0,0.0,0.213,0.000,0.0,0.000,0.213
11938494,9606.ENSP00000485678,9606.ENSP00000308270,0.0,0.0,0.0,0.152,0.000,0.0,0.000,0.151
11938495,9606.ENSP00000485678,9606.ENSP00000335660,0.0,0.0,0.0,0.182,0.000,0.0,0.000,0.181
11938496,9606.ENSP00000485678,9606.ENSP00000300127,0.0,0.0,0.0,0.155,0.000,0.0,0.000,0.154


TypeError: cannot unpack non-iterable NoneType object